In [3]:
### Install required packages here
#!pip install imageio

### ConvLSTM Implementation

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM2D, BatchNormalization, Conv3D
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point
import imageio

In [5]:
# Load the CSV file
df = pd.read_csv(r"./csv/Bounds_HAI.csv")
df.head()

,AGG_DAY_PERIOD,BOUNDS,CENTROID_LAT,CENTROID_LON,MEAN_ACTIVITY_INDEX_TOTAL
0,2020-01-20,"(5.873565673828125, 51.04053042130972, 5.87493...",51.040962,5.874252,0.018629
1,2020-01-20,"(5.87493896484375, 51.02844005602988, 5.876312...",51.028872,5.875626,0.164530
2,2020-01-20,"(5.876312255859375, 51.03103111414404, 5.87768...",51.031463,5.876999,0.203132
3,2020-01-20,"(5.876312255859375, 51.04053042130972, 5.87768...",51.040962,5.876999,0.233522
4,2020-01-20,"(5.876312255859375, 51.04139389812637, 5.87768...",51.041826,5.876999,0.009507


In [7]:
# Convert date field to datetime
df['AGG_DAY_PERIOD'] = pd.to_datetime(df['AGG_DAY_PERIOD'])

# Normalize the MEAN_ACTIVITY_INDEX_TOTAL
scaler = MinMaxScaler()
df['MEAN_ACTIVITY_INDEX_TOTAL'] = scaler.fit_transform(df[['MEAN_ACTIVITY_INDEX_TOTAL']])

In [4]:
# Create a sequence of data for the model
sequence_length = 10  # Define the sequence length
sequences = []
for i in range(len(data) - sequence_length):
    sequences.append(data.iloc[i:i + sequence_length])

# Convert the list of sequences to a NumPy array
sequences = np.array(sequences)

# Split into train, validation, and test sets
train_data, test_data = train_test_split(sequences, test_size=0.2, shuffle=False)
train_data, val_data = train_test_split(train_data, test_size=0.25, shuffle=False)

In [ ]:
# Prepare the data for ConvLSTM
def prepare_data(data):
    X = []
    y = []
    for sequence in data:
        X.append(sequence[['CENTROID_LAT', 'CENTROID_LON', 'MEAN_ACTIVITY_INDEX_TOTAL']].values)
        y.append(sequence[-1]['MEAN_ACTIVITY_INDEX_TOTAL'])
    X = np.array(X)
    y = np.array(y)
    X = X.reshape((X.shape[0], X.shape[1], 1, X.shape[2], 1))
    return X, y

In [10]:
X_train, y_train = prepare_data(train_data)
X_val, y_val = prepare_data(val_data)
X_test, y_test = prepare_data(test_data)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM2D, BatchNormalization, Conv3D
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point
import imageio

# Load the CSV data
data = pd.read_csv('Bounds_Hai.csv')

# Convert date field to datetime
data['AGG_DAY_PERIOD'] = pd.to_datetime(data['AGG_DAY_PERIOD'])

# Normalize the MEAN_ACTIVITY_INDEX_TOTAL
scaler = MinMaxScaler()
data['MEAN_ACTIVITY_INDEX_TOTAL'] = scaler.fit_transform(data[['MEAN_ACTIVITY_INDEX_TOTAL']])

# Create a sequence of data for the model
sequence_length = 10  # Define the sequence length
sequences = []
for i in range(len(data) - sequence_length):
    sequences.append(data.iloc[i:i + sequence_length])

# Convert the list of sequences to a NumPy array
sequences = np.array(sequences)

# Split into train, validation, and test sets
train_data, test_data = train_test_split(sequences, test_size=0.2, shuffle=False)
train_data, val_data = train_test_split(train_data, test_size=0.25, shuffle=False)

# Prepare the data for ConvLSTM
def prepare_data(data):
    X = []
    y = []
    for sequence in data:
        X.append(sequence[['CENTROID_LAT', 'CENTROID_LON', 'MEAN_ACTIVITY_INDEX_TOTAL']].values)
        y.append(sequence[-1]['MEAN_ACTIVITY_INDEX_TOTAL'])
    X = np.array(X)
    y = np.array(y)
    X = X.reshape((X.shape[0], X.shape[1], 1, X.shape[2], 1))
    return X, y

X_train, y_train = prepare_data(train_data)
X_val, y_val = prepare_data(val_data)
X_test, y_test = prepare_data(test_data)


In [ ]:
# Build the ConvLSTM model
model = Sequential()
model.add(ConvLSTM2D(filters=64, kernel_size=(1, 3), input_shape=(sequence_length, 1, 3, 1), padding='same', return_sequences=True))
model.add(BatchNormalization())
model.add(ConvLSTM2D(filters=64, kernel_size=(1, 3), padding='same', return_sequences=True))
model.add(BatchNormalization())
model.add(ConvLSTM2D(filters=64, kernel_size=(1, 3), padding='same'))
model.add(BatchNormalization())
model.add(Conv3D(filters=1, kernel_size=(1, 1, 1), activation='sigmoid'))

model.compile(optimizer='adam', loss='mse')

model.summary()


In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=16, validation_data=(X_val, y_val))

# Evaluate the model
loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')


In [ ]:
# Generate a random CSV file
random_data = {
    'AGG_DAY_PERIOD': pd.date_range(start='2021-01-01', periods=100),
    'CENTROID_LAT': np.random.uniform(low=-90, high=90, size=100),
    'CENTROID_LON': np.random.uniform(low=-180, high=180, size=100),
    'MEAN_ACTIVITY_INDEX_TOTAL': np.random.uniform(low=0, high=1, size=100)
}
random_df = pd.DataFrame(random_data)
random_df.to_csv('random_data.csv', index=False)

# Preprocess the random data
random_sequences = []
for i in range(len(random_df) - sequence_length):
    random_sequences.append(random_df.iloc[i:i + sequence_length])

random_sequences = np.array(random_sequences)
X_random, _ = prepare_data(random_sequences)

# Predict using the trained model
predictions = model.predict(X_random)

# Rescale the predictions back to original scale
predictions_rescaled = scaler.inverse_transform(predictions.reshape(-1, 1)).flatten()

# Save the predictions to CSV
prediction_dates = random_df['AGG_DAY_PERIOD'][sequence_length:].reset_index(drop=True)
prediction_results = pd.DataFrame({'Date': prediction_dates, 'Predicted_Activity_Index': predictions_rescaled})
prediction_results.to_csv('predictions.csv', index=False)


In [ ]:
# Save predictions as a shapefile
gdf = gpd.GeoDataFrame(prediction_results, geometry=gpd.points_from_xy(random_df['CENTROID_LON'][sequence_length:], random_df['CENTROID_LAT'][sequence_length:]))
gdf.to_file('predictions.shp')


In [ ]:
# Generate animated map of the predictions
images = []
dates = prediction_results['Date'].unique()
for date in dates:
    daily_data = prediction_results[prediction_results['Date'] == date]
    plt.figure(figsize=(10, 6))
    plt.scatter(daily_data['geometry'].x, daily_data['geometry'].y, c=daily_data['Predicted_Activity_Index'], cmap='viridis')
    plt.colorbar(label='Predicted Activity Index')
    plt.title(f'Predicted Activity Index on {date.date()}')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.savefig(f'map_{date.date()}.png')
    images.append(imageio.imread(f'map_{date.date()}.png'))
    plt.close()

imageio.mimsave('predictions_animation.gif', images, duration=0.5)
